Welcome new ADAM user!

   This notebook will guide you through a few ADAM features and validate that your system is configured correctly.
   This guide will demonstate how to pull orbital data for a given asteroid and time frame from JPL Horizons and how to propogate that same orbit using ADAM.  This program will also allow the user to create plots to compare the orbital data from the two sources as well as provide multiple ways to export the data for other applications. 
    

   Before starting this notebook, you should have already run the config_demo.ipynb notebook on your system (located in adam_home/demos).  Additionally, please ensure that you have installed Astroquery on your system (instructions in FAQ).    
   
   
Currently, as of 25-MAR-19, this notebook supports the following features. 

-Pulls trajectory data for a given object and timeframe from NASA JPL

-Propagates trajectory for that same object in ADAM. 

-Plots orbits for ADAM and JPL data.  

-Exports ADAM Data in CSV format

-Exports JPL Data in CSV format

-Calculates and displays RIC for ADAM vs JPL

- Exports ADAM ephemeris as .e file. 


In Development

- Transform ADAM data to rotate from EMEME2000 to ICRF

- Add option to output JPL data as .e 


NOTES: 

-This version currently defaults to the dev server since, as of 07-MAR-19, the production side is pending updates.

-This version is written to read the astro folder from within "adam_home".  You may need to point the code in the next cell to your Astro folder.    


 


#  These first cells must be run in order any time you operate the notebook. 

# Define Functions (for demo purposes) (will be a seperate file in standard notebook)

In [ ]:
# Import Transform Functions
# These will later be in a seperate file

"""
    transform.py

    Sources:
    Vallado, David - Fundamentals of Astrodynamics and Applications
"""

import numpy as np
from math import sin, cos, acos, sqrt

TOL = 1.0e-10
TWO_PI = 2 * np.pi

class Transform(object):
    """
    Does all transformations, including between coordinate systems
    TODO: add unit tests
    """
    def __init__(self):
        """Initialize attributes
        """
        self._mu = 132712440018.0  # sun gravitational parameter, km^3/s^2

    def set_gravitational_parameter(self, mu):
        """
        Set the gravitational parameter of interest

        Args:
            mu (float) - standard gravitational parameter, km^3/s^2

        Returns:
            None
        """
        self._mu = mu

    def Rotx(self, angle):
        """
        Returns the transformation corresponding to the rotation about the x-axis
        by an angle.

        Args:
            angle (float) - angle in radians

        Returns:
            R (numpy matrix) - rotation matrix
        """
        cosangle = cos(angle)
        sinangle = sin(angle)

        # Return rotation matrix
        R = np.array([[1, 0, 0], \
                      [0, cosangle, sinangle], \
                      [0, -sinangle, cosangle]])
        return R

    def Roty(self, angle):
        """
        Returns the transformation corresponding to the rotation about the y-axis
        by an angle.

        Args:
            angle (float) - angle in radians

        Returns:
            R (numpy matrix) - rotation matrix
        """
        cosangle = cos(angle)
        sinangle = sin(angle)

        # Return rotation matrix:
        R = np.array([[cosangle, 0, -sinangle], \
                      [0, 1, 0], \
                      [sinangle, 0, cosangle]])
        return R

    def Rotz(self, angle):
        """
        Returns the transformation corresponding to the rotation about the z-axis
        by an angle.

        Args:
            angle (float) - angle in radians

        Returns:
            R (numpy matrix) - rotation matrix
        """
        cosangle = cos(angle)
        sinangle = sin(angle)

        # Return rotation matrix:
        R = np.array([[cosangle, sinangle, 0], \
                      [-sinangle, cosangle, 0], \
                      [0, 0, 1]])
        return R

    def classical_to_cartesian(self, a, e, i, node, w, TA):
        """ Convert classical elements to cartesian elements

        Args:
            a (float) = semi-major axis (km)
            e (float) = eccentricity
            i (float) = inclination (0-pi rad)
            node (float) = right ascension of ascending node (0-2pi rad)
            w (float) = argument of periapsis (0-2pi rad)
            TA (float) = true anomaly (0-2pi rad)

        Returns:
            r (numpy array) - cartesian position (km)
            v (numpy array) - cartesian velocity (km/s)
        """
        # Convert
        p = a * (1 - e ** 2)  # semi-latus rectum, km

        # Determine what type of orbit is involved and set up angles for special cases
        if e < TOL:
            # argument of periapsis and RAAN are 0 for circular orbits
            w = 0.0
            # Circular equatorial
            if (i < TOL) or (abs(i - np.pi) < TOL):
                node = 0.0
        # Elliptical equatorial
        elif (i < TOL) or (abs(i - np.pi) < TOL):
            node = 0.0

        # Position and velocity vectors in perifocal coordinate system
        r_pqw = [0.0] * 3
        v_pqw = [0.0] * 3
        temp = p / (1.0 + e * cos(TA))
        r_pqw[0] = temp * cos(TA)
        r_pqw[1] = temp * sin(TA)

        if abs(p) < 0.0001:
            p = 0.0001

        v_pqw[0] = -sin(TA) * sqrt(self._mu) / sqrt(p)
        v_pqw[1] = (e + cos(TA)) * sqrt(self._mu) / sqrt(p)

        # Transform to cartesian
        tempvec = self.Rotz(-w).dot(r_pqw)
        tempvec = self.Rotx(-i).dot(tempvec)
        r = self.Rotz(-node).dot(tempvec)

        tempvec = self.Rotz(-w).dot(v_pqw)
        tempvec = self.Rotx(-i).dot(tempvec)
        v = self.Rotz(-node).dot(tempvec)

        return r, v

    def cartesian_to_classical(self, r, v):
        """ Convert cartesian elements to classical elements

        Args:
            r (list or array) - cartesian position (km)
            v (list or array) - cartesian velocity (km/s)

        Returns:
            a (float) = semi-major axis (km)
            e (float) = eccentricity
            i (float) = inclination (0-pi rad)
            node (float) = right ascension of ascending node (0-2pi rad)
            w (float) = argument of periapsis (0-2pi rad)
            TA (float) = true anomaly (0-2pi rad)
        """

        # Ensure r, v are numpy arrays
        r = np.array(r)
        v = np.array(v)

        # Get magnitudes of r, v
        mag_r = np.linalg.norm(r)
        mag_v = np.linalg.norm(v)

        # Find h, n, and e vectors
        hbar = np.cross(r, v)  # angular momentum vector
        mag_h = np.linalg.norm(hbar)

        nbar = np.array([0.0] * 3)  # line of nodes
        ebar = np.array([0.0] * 3)  # eccentricity vector

        if mag_h > TOL:

            # Define line of nodes
            nbar[0] = -hbar[1]
            nbar[1] = hbar[0]
            mag_n = np.linalg.norm(nbar)
            c1 = mag_v ** 2 - self._mu / mag_r

            # Get eccentricity
            for i in range(3):
                ebar[i] = (c1 * r[i] - (r.dot(v)) * v[i]) / self._mu

            e = np.linalg.norm(ebar)

            # Get semi-major axis
            a = 0.5 * mag_v ** 2 - self._mu / mag_r
            if abs(a) > TOL:
                a = -self._mu / (2 * a)
            else:
                a = float('nan')

            # Get inclination
            i = acos(hbar[2] / mag_h)

            # Get right ascension of ascending node
            if mag_n > TOL:
                temp = nbar[0] / mag_n
                if abs(temp) > 1.:
                    temp = np.sign(temp)
                node = acos(temp)
                if nbar[2] <= 0.:
                    node = TWO_PI - node
            else:
                node = float('nan')

            # Get argument of periapsis and true anomaly
            if e > TOL:
                w = acos((nbar.dot(ebar)) / (mag_n * e))
                TA = acos((ebar.dot(r)) / (e * mag_r))
                if ebar[2] < 0.:
                    w = TWO_PI - w
                if r.dot(v) < 0.:
                    TA = TWO_PI - TA
            else:
                w = float('nan')
                TA = float('nan')
        else:
            a = float('nan')
            e = float('nan')
            i = float('nan')
            node = float('nan')
            w = float('nan')
            TA = float('nan')

        return a, e, i, node, w, TA
    

    def ICRF_to_EMEME2000(vec_iX,vec_iY,vec_iZ):
        # Convert the obliquity from arcseconds to degrees and then to radians for Python
        obliquity = np.deg2rad(84381.448/3600.0)
           
        # rotation angle phi
        phi = obliquity
        
        # The final (rotated) vector
        vec_fX  =  vec_iX
        vec_fY  =  vec_iY*np.cos(phi)+ vec_iZ*np.sin(phi)
        vec_fZ  = -(vec_iY*np.sin(phi)) + vec_iZ*np.cos(phi)

        return vec_fX, vec_fY, vec_fZ 


    def EMEME2000_to_ICRF(vec_iX,vec_iY,vec_iZ):
                # Convert the obliquity from arcseconds to degrees and then to radians for Python
        obliquity = np.deg2rad(84381.448/3600.0)
            
        # rotation angle phi 
        phi = -(obliquity) 
        
        # The final (rotated) vector
        vec_fX  =  vec_iX
        vec_fY  =  vec_iY*np.cos(phi)+ vec_iZ*np.sin(phi)
        vec_fZ  = -(vec_iY*np.sin(phi)) + vec_iZ*np.cos(phi)

        return vec_fX, vec_fY, vec_fZ 



In [ ]:
# Import Transform Functions
# These will later be in a seperate file

"""
    asteroid_database.py

    Wrapper for the astroquery JPL Horizons API
    Does all conversions and returns relevant information
"""

import astroquery
from astroquery.jplhorizons import Horizons

AU = 149597870.0     # conversion from AU to km
DAY2SEC = 24*60*60   # conversion from days to seconds

class AsteroidDB(object):
    """
    Grabs asteroid ephemerides from JPL Horizons database
    TODO: add orbital elements return, unit tests
    """

    def __init__(self):
        """Initialize attributes
        """
        self._step = 1           # step size
        self._step_unit = 'day'  # step size unit

    def set_step_size(self, step, step_unit):
        """
        Set step size and step size unit
        Unit options are:
        sec, min, day, hour, year

        Args:
            step (int) - step size
            step_unit (str) - unit for step size

        Returns:
            None
        """
        options = ['sec', 'min', 'day', 'hour', 'year']

        if step_unit not in options:
            raise KeyError("Invalid units. Options are: %s" % options)

        # Enforce integer step
        step = int(step)

        self._step = step
        self._step_unit = step_unit

    def get_cartesian_states(self, obj_id, epoch, end_time):
        """
        Query database and get cartesian rv
        NOTE: id requires the Horizons name, not SPK number!

        Args:
            id (str) - object ID (Horizons name)
            epoch (datetime obj) - start time
            end_time (datetime obj) - end time

        Returns:
            rv (list of rv list) - rv in the format:
                x (km), y (km), z (km), vx (km/s), vy (km/s), vz (km/s)
        """
        # Format step size for query
        if self._step_unit == 'sec':
            # Special formatting needed
            dt = int((end_time - epoch).total_seconds()/self._step)
        else:
            # Concatenate step size and unit
            dt = str(self._step)+self._step_unit[0]

        # Make call to JPL Horizons database and get data
        try:
            obj = Horizons(id=obj_id, location='@sun',
                           epochs={'start': epoch.isoformat(),
                                   'stop': end_time.isoformat(),
                                   'step': dt})
            data = obj.vectors()
        except:
            raise RuntimeError("Invalid query")

        # Get column header
        columns = data.colnames

        # Make state vectors
        rv = []
        for row in data:
            r_mult = AU          # convert AU to km
            v_mult = AU/DAY2SEC  # convert AU/day to km/s

            rv.append([row[columns.index("x")]*r_mult,
                       row[columns.index("y")]*r_mult,
                       row[columns.index("z")]*r_mult,
                       row[columns.index("vx")]*v_mult,
                       row[columns.index("vy")]*v_mult,
                       row[columns.index("vz")]*v_mult])
        return rv

# This imports libraries from ADAM and Python

In [ ]:
# This specifies the path to the adam_home folder and configuration file on your system.
import sys

import adam

# This imports the necessary libraries from ADAM and JPL as well as other packages. 
from adam import Batch
from adam import Batches
from adam import BatchRunManager
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import ProjectsClient
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os
from adam import Service
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import ConfigManager
import unittest
import datetime
import os
import numpy as np
import numpy.testing as npt
from adam.batch import StateSummary
from adam.batch import PropagationResults

# Astro Query
from astroquery.jplhorizons import Horizons
from astroquery.jplhorizons import conf
conf.horizons_server = 'https://ssd.jpl.nasa.gov/horizons_batch.cgi'



#NOTE: This version is written to read the astro folder from within "adam_home". You may need to move astro to this folder. 
#If the astro folder is not in adam_home, please modify the next line to point to your "astro" folder. 
#sys.path.insert(0,'C:\\Users\\Lowell\\adam_home\\astro') 
#astro Lib


# These will be kept when calling astro from its own folder
#from astro import Transform
#from astro import asteroid_database
  


#Other Lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 


# Define Constants
GM = 132712440041.93938   #From JPL Horizons
AU_Meters = 1.49597870e+11  # Astronomical Units in meters  
AU_Km = 149597870.0     # conversion from AU to km 
AU_per_day = 1731456.84 # AU/Day converted to Meters per second
# From NASA JPL.  The astronomical unit (au) is defined by the IAU as exactly 149,597,870,700 m.

# Identify Asteroid and time frame, pull data from JPL

In [ ]:
print("Enter asteroid target number. e.g for Eros use 433. IDs can be searched here: https://ssd.jpl.nasa.gov/sbdb.cgi#top ")
asteroid = input() #in form of target number i.e. for Eros use 433  #10 Hygiea

# You can edit the inputs below to change the start/stop times and the step size. 
start_time_str='2016-10-04T00:00:00' #YYYY-Mon-Dy {HH:MM}   #Default: start_time_str='2016-10-04T00:00:00'
end_time_str='2017-10-11T00:00:00'   # Default: end_time_str='2017-10-11T00:00:00'
step = '1d'   #Default:  1d

In [ ]:
#call the object's data from JPL

# Original
obj = Horizons(id=str(asteroid), location=None ,epochs={'start':start_time_str, 'stop':end_time_str,'step':step})
vec = obj.vectors() #state vector at each epoch, inital state vector is initial epoch
#vec is the entire epheremris over the specified timeframe, the inital state vector (to be put into ADAM) is vec's first row

#call the object's data from JPL  # Fails when sent to ADAM
# For Geocentric TESTING
#obj = Horizons(id=str(asteroid), location='Geocentric'  ,epochs={'start':start_time_str, 'stop':end_time_str,'step':step})
#vec = obj.vectors() #state vector at each epoch, inital state vector is initial epoch
# tried location='Geocentric'   Didn't work when sending to ADAM
# Trying location=500   # Fails when submitting to ADAM

In [ ]:
#Simplified Dataframe

eph_JPL_df = pd.DataFrame(
    {'Time [Barycentric TBD]': vec['datetime_jd'],
     'x [m]': vec['x']*AU_Meters,#1.496e+11,
     'y [m]': vec['y']*AU_Meters,#1.496e+11,
     'z [m]': vec['z']*AU_Meters,#1.496e+11,
     'x_dot [m/s]': vec['vx']*AU_per_day,#1731456.84,
     'y_dot [m/s]': vec['vy']*AU_per_day ,#1731456.84,
     'z_dot [m/s]': vec['vz']*AU_per_day, #1731456.84
     
    })
# eph_JPL_df    # Commented out to improve readibility. 

In [ ]:
#Organize JPL Data to submit state vector to ADAM. 

#JPL Cartesian State Vector + Data Frame
JPL_SV = np.array([vec[i][0] for i in vec.columns])
JPL_SV_df_cart = pd.DataFrame(JPL_SV, index= vec.columns, columns=list(' ')) 
#JPL r and v vectors
JPL_r = np.array([np.float(i) for i in JPL_SV[5:8]])*AU_Km#1.496e+8
JPL_v = np.array([np.float(i) for i in JPL_SV[8:11]])*(AU_per_day/1000) # KM per second  1731.46
print("Position: %s km" % JPL_r)
print("Velocity: %s km/s" % JPL_v)
#########################################################################
#Converting to Keplerian 
d2r = np.pi/180.
r2d = 180./np.pi
trans = Transform()

JPL_a, JPL_e, JPL_i, JPL_node, JPL_w, JPL_TA = trans.cartesian_to_classical(JPL_r,JPL_v)
kep_col = np.array(['SMA','Ecc','Inc','RAAN','Arg of per','True anomaly'])

JPL_SV_df_kep = pd.DataFrame(np.array([JPL_a, JPL_e, JPL_i*r2d, JPL_node*r2d, JPL_w*r2d, JPL_TA*r2d]), index= kep_col, columns=list(' ')) 
#Combining two dataframes
JPL_SV_df = pd.concat([JPL_SV_df_cart, JPL_SV_df_kep], axis=0) 
# JPL_SV_df  # Again, commented out to avoid excessive outputs. 

In [ ]:
# Import Minor Planet Center Queries

#Redundant?

#from astroquery.mpc import MPC
#mpc = MPC()
#result = mpc.query_object_async(number=asteroid, target_type="asteroid")


#result.json()   # Remove # to see results
#[{'absolute_magnitude': '3.34', 'aphelion_distance': '2.976', 'arc_length': 79247, 'argument_of_perihelion': '73.11528', 'ascending_node': '80.3099167', 'critical_list_numbered_object': False, 'delta_v': 10.5, 'designation': None, 'earth_moid': 1.59353, 'eccentricity': '0.0755347', 'epoch': '2018-03-23.0', 'epoch_jd': '2458200.5', 'first_observation_date_used': '1801-01-31.0', 'first_opposition_used': '1801', 'inclination': '10.59351', 'jupiter_moid': 2.09509, 'km_neo': False, 'last_observation_date_used': '2018-01-20.0', 'last_opposition_used': '2018', 'mars_moid': 0.939285, 'mean_anomaly': '352.23052', 'mean_daily_motion': '0.2141308', 'mercury_moid': 2.18454, 'name': 'Ceres', 'neo': False, 'number': 1, 'observations': 6689, 'oppositions': 114, 'orbit_type': 0, 'orbit_uncertainty': '0', 'p_vector_x': '-0.87827466', 'p_vector_y': '0.33795664', 'p_vector_z': '0.33825868', 'perihelion_date': '2018-04-28.28378', 'perihelion_date_jd': '2458236.78378', 'perihelion_distance': '2.5580384', 'period': '4.6', 'pha': False, 'phase_slope': '0.12', 'q_vector_x': '-0.44248615', 'q_vector_y': '-0.84255514', 'q_vector_z': '-0.30709419', 'residual_rms': '0.6', 'saturn_moid': 6.38856, 'semimajor_axis': '2.7670463', 'tisserand_jupiter': 3.3, 'updated_at': '2018-02-26T17:29:46Z', 'uranus_moid': 15.6642, 'venus_moid': 1.84632}]
#MPC Eph database
#https://minorplanetcenter.net/iau/MPEph/MPEph.html?format=json

# Propagate through ADAM

In [ ]:
# TODO: just link to a notebook that does propagation

# Sets the current ADAM config environment.
config_manager = ConfigManager()
config_manager.set_default_env('dev')

config = config_manager.get_config()
service = Service(url=config['url'], project=config['workspace'])
service.setup()
auth_rest = AuthenticatingRestProxy(RestRequests())

In [ ]:
working_project = service.new_working_project() # cleans up old batches

In [ ]:
#Set Keplerian Parameters
start_time_str_Z=start_time_str+'Z'
end_time_str_Z=end_time_str+'Z'

#keplerian Method:
keplerian_elements = {
    'semi_major_axis_km': np.float(JPL_SV_df.iat[14,0]),
    'eccentricity': np.float(JPL_SV_df.iat[15,0]),
    'inclination_deg': np.float(JPL_SV_df.iat[16,0]),
    'ra_of_asc_node_deg': np.float(JPL_SV_df.iat[17,0]),
    'arg_of_pericenter_deg': np.float(JPL_SV_df.iat[18,0]),
    'true_anomaly_deg': np.float(JPL_SV_df.iat[19,0]),
    'gm': 132712440041.93938 # FROM JPL Horizons                                    
}
propagation_params = PropagationParams({
    'start_time': start_time_str_Z,
    'end_time': end_time_str_Z,
    'project_uuid': config['workspace'],
    'description': 'Created by test at ' + start_time_str
})
opm_params_templ = {
    'epoch': start_time_str_Z,
    'object_name': (JPL_SV_df.iat[0,0]),    # object ID
    'object_id':asteroid
    #'comment':"Keplerian Coord System"                    
    # state_vector or keplerian_elements will be set later.
    #'mass': 500.5,
    #'solar_rad_area': 25.2,
    #'solar_rad_coeff': 1.2,
    #'drag_area': 33.3,
    #'drag_coeff': 2.5
}
###################################################################################

### Eventually we want to define and call these functions from elsewhere. 
#Create keplerian batch files to be put into ADAM
def make_keplerian_batches(start_time_str, end_time_st):
    keplerian_opm_params = opm_params_templ.copy()
    keplerian_opm_params['keplerian_elements'] = keplerian_elements

    keplerian = Batch(propagation_params, OpmParams(keplerian_opm_params))
    return  keplerian


kep_batch= make_keplerian_batches(start_time_str, end_time_str)
print(kep_batch.get_opm_params().generate_opm())

# Submit and wait until batch run is ready
batches_module = Batches(auth_rest)
BatchRunManager(batches_module, [kep_batch]).run()

###################################################################################
# Retrieve the vector at the end of the state
def kep_single_run(self, start_time_str, end_time_str):
    
    #start_time_str = start.isoformat() + 'Z'
    #end_time_str = end.isoformat() + 'Z'

    keplerian = make_keplerian_batches(start_time_str, end_time_str) #batch objets
    print(keplerian.get_state_summary().get_parts_count())

    BatchRunManager(self.service.get_batches_module(), [kep_batch]).run()

    keplerian_end_state = kep_batch.get_results().get_end_state_vector()
    return keplerian_end_state

# Get the end state vector 
end_state_vector = kep_batch.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector)
###################################################################################
#Get status and parts count
parts_count = kep_batch.get_state_summary().get_parts_count()
print("Final state: %s, part count %s\n" % (kep_batch.get_calc_state(), parts_count))

In [ ]:
# Pull ephemeris from ADAM results and organize into data frames.
# TODO: delete this section in favor of https://github.com/B612-Asteroid-Institute/adam_home/blob/master/demos/Loading%20Run%20Data%20Example.ipynb
part_to_get = 0
eph = kep_batch.get_results().get_parts()[part_to_get].get_ephemeris()
print(eph)

_ ,eph_temp = eph.split("EphemerisTimePosVel\n\n", 1)
eph_temp, _ = eph_temp.split("\n\nEND Ephemeris", 1)
eph_temp

eph_temp_split=eph_temp.split(' ')
#eph_temp_split

eph_split_fix=[]
atomic_times=[]
atomic_times.append(np.float(eph_temp_split[0]))
for i in range(1, len(eph_temp_split[0::6])-1):
    #calculate atomic time
    temp = eph_temp_split[0::6]
    temp_2=temp[i]
    #print(temp_2)
    eph_split_again,atomic_time= temp_2.split('\n')
    atomic_times.append(np.float(atomic_time))
    eph_split_fix.append(np.float(eph_split_again))
eph_split_fix.append(np.float(eph_temp_split[-1]))


eph_split_fix.insert(0,0)
#eph_split_fix.append(0)

# Can we omit this?
eph_temp_split[0::6]=eph_split_fix

for i in range(0,len(eph_temp_split)):
    eph_temp_split[i]=np.float(eph_temp_split[i])

eph_Adam=eph_temp_split[1:len(eph_temp_split)]

#changeeph_Adam from long list to list of 6 elements of statevector
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

eph_Adam_vectors=[]
for group in chunker(eph_Adam, 6):
    eph_Adam_vectors.append((group))
    
eph_Adam_x=[]
eph_Adam_y=[]
eph_Adam_z=[]
eph_Adam_xdot=[]
eph_Adam_ydot=[]
eph_Adam_zdot=[]
for i in range(0,len(eph_Adam_vectors)):
    temp=eph_Adam_vectors[i]
    eph_Adam_x.append(temp[0])
    eph_Adam_y.append(temp[1])
    eph_Adam_z.append(temp[2])
    eph_Adam_xdot.append(temp[3])
    eph_Adam_ydot.append(temp[4])
    eph_Adam_zdot.append(temp[5])
#############################################################################

#Construct ADAM dataframe for the JPL state vector
eph_Adam_vectors = pd.DataFrame(
    {'Time [Atomic UTC]': atomic_times,
     'x [m]': eph_Adam_x,
     'y [m]': eph_Adam_y,
     'z [m]': eph_Adam_z,
     'x_dot [m/s]': eph_Adam_xdot,
     'y_dot [m/s]': eph_Adam_ydot,
     'z_dot [m/s]': eph_Adam_zdot
     
    })

 # This line below constructs a seperate dataframe for RIC calculations.  
 # It's somewhate redundant, but helps keep track of which data we're using.  
eph_JPLtoAdam_df = pd.DataFrame(
    {'Time [Atomic UTC]': atomic_times,
     'x [m]': eph_Adam_x,
     'y [m]': eph_Adam_y,
     'z [m]': eph_Adam_z,
     'x_dot [m/s]': eph_Adam_xdot,
     'y_dot [m/s]': eph_Adam_ydot,
     'z_dot [m/s]': eph_Adam_zdot
     
    })

# (At this point, you can skip to any function in the notebook.)

# Plot Orbits for JPL and ADAM.

In [ ]:
# Visualize ADAM and JPL orbits for the object.

# Let's just do Earth and object for now, this could be expanded to show other planets. 
earth = Horizons(id='Geocenter', location=None, epochs={'start':start_time_str, 'stop':end_time_str, 'step':'1d'}, id_type = 'majorbody')
vEarth = earth.vectors()
#sun = Horizons(id='Sun', location=None, epochs={'start':start_time_str, 'stop':end_time_str, 'step':'1d'}, id_type = 'majorbody')
#vSun = sun.vectors()

#  Get the points for JPL object   
n = len(eph_JPL_df)
xObj = [1]*n               # initialize X position array
yObj = [1]*n               # initialize Y position array
zObj = [1]*n               # initialize Z position array


for i in range(0,n):             
            xObj[i] = eph_JPL_df['x [m]'][i]
            yObj[i] = eph_JPL_df['y [m]'][i]
            zObj[i] = eph_JPL_df['z [m]'][i]           
            
#  Get the points for ADAM object            
n = len(eph_Adam_vectors)           
xADAM_Obj = [1]*n               # initialize X position array
yADAM_Obj = [1]*n               # initialize Y position array
zADAM_Obj = [1]*n               # initialize Z position array
for i in range(0,n):             
            xADAM_Obj[i] = eph_Adam_vectors['x [m]'][i]
            yADAM_Obj[i] = eph_Adam_vectors['y [m]'][i]
            yADAM_Obj[i] = eph_Adam_vectors['z [m]'][i]
            
# Earth points  
n = len(vEarth)
xEarth = [1]*n               # initialize X position array
yEarth = [1]*n               # initialize Y position array
zEarth = [1]*n               # initialize Z position array
for i in range(0,n):             
            xEarth[i] = (vEarth['x'][i]*AU_Meters)
            yEarth[i] = (vEarth['y'][i]*AU_Meters)
            zEarth[i] = (vEarth['z'][i]*AU_Meters)

# (In Development) Transform ADAM Vectors from EMEME2000 to ICRF


In [ ]:
n = len(eph_Adam_vectors) #len(ADAM_ICRF)    


#Initialized transformed values
TxvUdo = []             # initialize X position array
TyvUdo = []              # initialize Y position array
TzvUdo = []              # initialize Z position array

for i in range (n):
    # Transforms ADAM positions from EMEME2000 to ICRF
    eph_ADAM_ICRF_temp = Transform.EMEME2000_to_ICRF(eph_Adam_vectors['x [m]'][i],eph_Adam_vectors['y [m]'][i],eph_Adam_vectors['z [m]'][i])       
    #eph_ADAM_ICRF_temp = Transform.ICRF_to_EMEME2000(eph_Adam_vectors['x [m]'][i],eph_Adam_vectors['y [m]'][i],eph_Adam_vectors['z [m]'][i])        
    # Pulls, X, Y, and Z values   
    TxvUdo.append(eph_ADAM_ICRF_temp[0])
    TyvUdo.append(eph_ADAM_ICRF_temp[1])
    TzvUdo.append(eph_ADAM_ICRF_temp[2])
    
    
    #eph_Adam_vectors['x [m/s]'][i] = eph_ADAM_ICRF_temp[0]
    #eph_Adam_vectors['y [m/s]'][i] = eph_ADAM_ICRF_temp[1]
    #eph_Adam_vectors['z [m/s]'][i] = eph_ADAM_ICRF_temp[2]
    # For now, we are transforming the coordinates and recording a new variable so that we can compare/troubleshoot.  
    #In the future it would be simpler to replace the values as they're transformed, as in the velocity example below.  


# Transform velocities also
    eph_ADAM_ICRF_temp1 = Transform.EMEME2000_to_ICRF(eph_Adam_vectors['x_dot [m/s]'][i],eph_Adam_vectors['y_dot [m/s]'][i],eph_Adam_vectors['z_dot [m/s]'][i])       
    # Pulls, X, Y, and Z velocity values   
    #eph_ADAM_ICRF_temp1 = Transform.ICRF_to_EMEME2000(eph_Adam_vectors['x_dot [m/s]'][i],eph_Adam_vectors['y_dot [m/s]'][i],eph_Adam_vectors['z_dot [m/s]'][i])       
    
    eph_Adam_vectors['x_dot [m/s]'][i] = eph_ADAM_ICRF_temp1[0] 
    eph_Adam_vectors['y_dot [m/s]'][i] = eph_ADAM_ICRF_temp1[1] 
    eph_Adam_vectors['z_dot [m/s]'][i] = eph_ADAM_ICRF_temp1[2] 

In [ ]:
#  Plot ADAM and JPL 
  
plot_size = max([np.abs(xObj).max() ]) 
size = [-plot_size-(plot_size*0.10), plot_size+(plot_size*0.10)]  
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection ='3d')   

# Plot JPL Object
ax.plot(xObj,yObj,zObj, c = 'orange')

#Plot ADAM Object
ax.plot(xADAM_Obj,yADAM_Obj,zADAM_Obj,c='red')  

#Plot ADAM Trans-to_ICRF Object
ax.plot(TxvUdo,TyvUdo,TzvUdo,c='purple')   # In Development

#Plot Earth
ax.plot(xEarth,yEarth,zEarth,c='blue')


ax.text(0.5*plot_size,0.5*plot_size,0.8*plot_size, asteroid + " JPL",color='orange')
ax.text(0.5*plot_size,0.5*plot_size,0.5*plot_size, "Earth",color='blue')
ax.text(0.5*plot_size,0.5*plot_size,0.2*plot_size, asteroid + " ADAM",color='red')
ax.text(0.5*plot_size,0.5*plot_size,1.1*plot_size, asteroid + " ADAM Transformed",color='purple')
#ax.text(0,1.5,2.3, asteroid + " ADAM in ICRF",color='purple')   # In development

ax.set_xlim(size), ax.set_ylim(size), ax.set_zlim(size)
plt.show()

# The following cells allow you to export the ADAM data to .csv and STK formats. (Can be run multiple times)

In [ ]:
#Export ADAM or JPL data to CSV(Excel)

OutputMethod = input(    'Enter 1 to export ADAM data, enter 2 to export JPL Data')
OutputMethod = int(OutputMethod)
print("This will default export location to your adam_home folder, you can modify this in the cell") 
CSV_name = input("Enter file name")
CSV_export  = adam_home_defined + "/" +CSV_name+'.csv'

if OutputMethod == 1 :
    #pd.DataFrame(eph_ADAM_1).to_csv(CSV_expd.DataFrame(eph_Adam_vectors).to_csv(CSV_export)port)
    pd.DataFrame(eph_Adam_vectors).to_csv(CSV_export)  # Export Adam file  
elif OutputMethod ==2 :
    pd.DataFrame(eph_JPL_df).to_csv(CSV_export)     #Export JPL Data
else: 
    print("Input not recognized")
# Note:  This currently pulls the ADAM data prior to coordinate transform.  
# Once that function is developed further we'll need to call those data

# Output Ephemeris to STK (In Development)

Skip to last cell if you only want to export a .e file without opening STK


In [ ]:
#print("Ephemeris:")
#print(eph)

In [ ]:
#from win32api import GetSystemMetrics
#from comtypes.client import CreateObject
# Start STK on the desktop (on Microsoft Windows)
#uiApplication=CreateObject("STK11.Application")
#uiApplication.Visible=True
#uiApplication.UserControl=True

#root = uiApplication.Personality2
#stk_app = win.gencache.EnsureDispatch('STK11.Application')
#stk_app.Visible = True
#print(stk_app.__module__)

#STK = stk_app.Personality
#root = stk_app.Personality2
#from comtypes.gen import STKUtil
#from comtypes.gen import STKObjects

In [ ]:
#root.NewScenario("ADAM_Starter")
#scenario = root.CurrentScenario

In [ ]:
#Set Scenario Time   If you changed default times above you'll need to manually format them here. 
#scenario2 = scenario.QueryInterface(STKObjects.IAgScenario)
#scenario2.SetTimePeriod('04 Oct 2016 16:00:05.000', '04 Oct 2017 16:00:05.000')
#root.Rewind();
#Default Start times
#start_time_str='2016-10-04T00:00:00' #YYYY-Mon-Dy {HH:MM}   #Default: start_time_str='2016-10-04T00:00:00'
#end_time_str='2017-10-11T00:00:00' 

In [ ]:
# From AGI Help
#3. Add a Satellite object to the scenario.
#satellite = scenario.Children.New(STKObjects.eSatellite, "LeoSat")
#4. Propagate the Satellite object's orbit.
#root.ExecuteCommand('SetState */Satellite/LeoSat Classical TwoBody "' + scenario2.StartTime + '" "'+ scenario2.StopTime +'" 60 ICRF "'+ scenario2.StartTime + '" 7200000.0 0.0 90 0.0 0.0 0.0')

In [ ]:
# Use STK MTO (Multi Track Object) to store epehmeris points in STK
#mto = root.CurrentScenario.Children('ADAM_Starter')
#root.ExecuteCommand('VO */MTO/AsteroidTracks System "CentralBody/Sun Inertial"')
#newMTO = mto.CopyObject('Temp') # satName)

    # Add ephemeris file as MTO track
#    newTrack = mto.Tracks.Add(countNum)
#    newTrack.Points.LoadPoints(os.getcwd() + '/' + fileName)
    
    # Set graphics properties of track
 #   newTrack.Interpolate = True
  #  mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Width = 1
   # mto.Graphics.Tracks.GetTrackFromId(countNum).Color = 0xe16941 # Blue Green Red: 225 105 065
  #  mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Translucency = 33

In [ ]:
#Skip to here to export .e file without opening STK. 

# Save ephemeris to STK ".e" files, and add as track to MTO object in STK
print("This saves a .e file to the adam_home folder")
print("WARNING: This will over-write a file with the same name")
countNum = input("Enter file number to save to STK format, (as an integer)")
countNum = int(countNum)
satName = asteroid
satName = str(satName)
    #Write ephemeris to file
#eph = b.get_results().get_parts()[0].get_ephemeris()
satName = '{:04d}'.format(countNum)
fileName = '{}.e'.format(satName)
print('STK satellite object name: {}    Ephemeris File Name: {}'.format(satName,fileName))
with open(fileName, 'w') as f:
        f.write(eph)
       
    
    # Add ephemeris file as MTO track
#newTrack = mto.Tracks.Add(countNum)
#newTrack.Points.LoadPoints(os.getcwd() + '/' + fileName)
    
    # Set graphics properties of track
#newTrack.Interpolate = True
#mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Width = 1
#mto.Graphics.Tracks.GetTrackFromId(countNum).Color = 0xe16941 # Blue Green Red: 225 105 065
#mto.Graphics.Tracks.GetTrackFromId(countNum).Line.Translucency = 33
    
    # Increment count for unique ephemeris and STK satellite object name
#countNum = countNum + 1

# Plot RIC for JPL vs ADAM  (Requires Validation after coordinate transfer is functional)

In [ ]:
# Transform ADAM coordinates from EMEME2000 to ICRF Coordinate System
n = len(eph_Adam_vectors)

for i in range (n):
    # Transforms ADAM positions from EMEME2000 to ICRF
    eph_ADAM_ICRF_temp1 = Transform.EMEME2000_to_ICRF(eph_JPLtoAdam_df['x [m]'][i],eph_JPLtoAdam_df['y [m]'][i],eph_JPLtoAdam_df['z [m]'][i])       
    # Pulls, X, Y, and Z values   
    eph_JPLtoAdam_df['x [m]'][i] = eph_ADAM_ICRF_temp1[0] 
    eph_JPLtoAdam_df['y [m]'][i] = eph_ADAM_ICRF_temp1[1] 
    eph_JPLtoAdam_df['z [m]'][i] = eph_ADAM_ICRF_temp1[1] 

# Transform velocities also
    eph_ADAM_ICRF_temp2 = Transform.EMEME2000_to_ICRF(eph_JPLtoAdam_df['x_dot [m/s]'][i],eph_JPLtoAdam_df['y_dot [m/s]'][i],eph_JPLtoAdam_df['z_dot [m/s]'][i])       
    # Pulls, X, Y, and Z values   
    eph_JPLtoAdam_df['x_dot [m/s]'][i] = eph_ADAM_ICRF_temp2[0] 
    eph_JPLtoAdam_df['y_dot [m/s]'][i] = eph_ADAM_ICRF_temp2[1] 
    eph_JPLtoAdam_df['z_dot [m/s]'][i] = eph_ADAM_ICRF_temp2[1] 

In [ ]:
# Original  
# Organize data from JPL and ADAM for RIC comparisions.  

# Need to confirm these calcuations once coordinate transdform is fixed. 
JPL_R = np.array([np.array(eph_JPL_df.iloc[i,4:7])/np.linalg.norm(eph_JPL_df.iloc[i,4:7]) for i in range(0,len(eph_JPL_df))])
JPLtoADAM_R = np.array([np.array(eph_JPLtoAdam_df.iloc[i,1:4])/np.linalg.norm(eph_JPLtoAdam_df.iloc[i,1:4]) for i in range(0,len(eph_JPLtoAdam_df))])

##
JPL_ADAM_RIC_R = np.array([(JPLtoADAM_R[i]-JPL_R[i])/np.linalg.norm(JPLtoADAM_R[i]-JPL_R[i]) for i in range(0,len(JPL_R))])
#print(JPL_ADAM_RIC_R)
JPL_V = np.array([np.array(eph_JPL_df.iloc[i,4:7])/np.sqrt(np.sum(np.square(eph_JPL_df.iloc[i,4:7]))) for i in range(0,len(eph_JPL_df))])

JPLtoADAM_V = np.array([np.array(eph_JPLtoAdam_df.iloc[i,4:7])/np.sqrt(np.sum(np.square(eph_JPLtoAdam_df.iloc[i,4:7]))) for i in range(0,len(eph_JPLtoAdam_df))])

##
JPL_ADAM_RIC_V = np.array([(JPLtoADAM_V[i]-JPL_V[i])/np.linalg.norm(JPLtoADAM_V[i]-JPL_V[i]) for i in range(0,len(JPL_R))])
#print(JPL_ADAM_RIC_V)

JPL_ADAM_RIC_C = np.array([np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_V[i]/np.linalg.norm(np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_V[i]))) for i in range(0,len(JPL_R))])
#JPL_ADAM_RIC_C

JPL_ADAM_RIC_I = np.array([np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_C[i]/np.linalg.norm(np.cross(JPL_ADAM_RIC_R[i],JPL_ADAM_RIC_C[i]))) for i in range(0,len(JPL_R))])
#JPL_ADAM_RIC_I

In [ ]:
atomic_times=atomic_times[0:len(atomic_times)-1]
plt.figure(figsize=(10,5))
plt.grid(True,ls='--')
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,0],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,0],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,0],linewidth=3,label="In Track")
plt.title("RIC for JPL vs ADAM (along x)")
plt.xlabel('Atomic Time')
# confirm?  needs units
plt.ylabel('Error in Position')
plt.legend()

plt.figure(figsize=(10,5))
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,1],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,1],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,1],linewidth=3,label="In Track")
plt.grid(True)
plt.title("RIC for JPL vs ADAM (along y)")
plt.xlabel('Atomic Time')
# confirm?  needs units
plt.ylabel('Error in Position')
plt.legend()

plt.figure(figsize=(10,5))
plt.plot(atomic_times,JPL_ADAM_RIC_R[:,2],linewidth=3,label="Radial")
plt.plot(atomic_times,JPL_ADAM_RIC_I[:,2],linewidth=3,label="Cross Track")
plt.plot(atomic_times,JPL_ADAM_RIC_C[:,2],linewidth=3,label="In Track")
plt.title("RIC for JPL vs ADAM (along z)")
plt.grid(True)
plt.xlabel('Atomic Time')
# confirm?  needs units
plt.ylabel('Error in Position')
plt.legend()



In [ ]:
#len(atomic_times[0:len(atomic_times)-1])   # for troubleshooting

In [ ]:
plt.show()

# What is the scale of vertical axis?